In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
!ls /content/drive/My\ Drive/

In [ ]:
!pip install ta

In [ ]:

import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
import ta
from ta import add_all_ta_features
from ta.utils import dropna
template = "plotly_dark"

In [ ]:
# Manual Download https://forexsb.com/historical-forex-data -> no volume

In [ ]:
import hashlib
import random

def color_from_time(time_value):
    """
    Generate a consistent random HEX color string based on a datetime or string.
    """
    if pd.isna(time_value):
        return "#cccccc"  # default gray for NaT

    # Convert datetime to string and hash it
    time_str = str(time_value)
    seed = int(hashlib.md5(time_str.encode()).hexdigest(), 16)

    # Seed RNG and generate color
    random.seed(seed)
    return '#{:02X}{:02X}{:02X}'.format(
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)
    )

In [ ]:
profit_color_continuous_scale =["red","orange" ,"White","lightgreen","green"]

In [ ]:
## Historical Order Data
raw_df = pd.read_excel('/content/drive/MyDrive/algo/input/ReportHistory-32012857.xlsx',skiprows=6)
raw_df = raw_df[~raw_df['Commission'].isnull()]
time_config = {
     "format":'%Y.%m.%d %H:%M:%S',
     "errors":'coerce'
}
# raw_df['Time.1'] = pd.to_datetime(raw_df['Time.1'], format='%Y.%m.%d %H:%M:%S', errors='coerce')
raw_df["Open Time"] = pd.to_datetime(raw_df["Time"],**time_config)
raw_df["Close Time"] = pd.to_datetime(raw_df["Time.1"],**time_config)
raw_df = raw_df.dropna(subset=['Time.1'])
raw_df["Volume"] = pd.to_numeric(raw_df["Volume"], errors="coerce")


raw_df = raw_df[raw_df['Open Time']>='2025-04-13']
raw_df = raw_df[raw_df['Close Time']<'2025-04-17']

# Step 2: Convert to UTC
# raw_df["Open Time"] = raw_df["Open Time"].dt.tz_localize("Asia/Bangkok")
# raw_df["Close Time"] = raw_df["Close Time"].dt.tz_localize("Asia/Bangkok")
# raw_df["Open Time"] = raw_df["Open Time"].dt.tz_convert("UTC")
# raw_df["Close Time"] = raw_df["Close Time"].dt.tz_convert("UTC")
raw_df["Open Time"] = raw_df["Open Time"] - pd.Timedelta(hours=3)
raw_df["Close Time"] = raw_df["Close Time"] - pd.Timedelta(hours=3)

raw_df["Open Time (15m)"] = raw_df["Open Time"].dt.floor("15min")
order_counts = raw_df.groupby("Open Time (15m)").size().reset_index(name="Orders in 15m")
raw_df = raw_df.merge(order_counts, on="Open Time (15m)", how="left")
# Drop rows where Volume is NaN (invalid)
raw_df = raw_df[raw_df["Volume"].notna()]
raw_df = raw_df.rename(columns={
    "Price": "Open Price",
    "Price.1": "Close Price",
    "Profit": "Profit USD"
})
dtype_mapping = {

    "Profit USD":'int64'
}
raw_df = raw_df.astype(dtype_mapping)
# Drop irrelevant or unnamed columns
df = raw_df[["Open Time (15m)","Orders in 15m","Open Time", "Close Time", "Symbol", "Type", "Volume", "Open Price", "Close Price", "T / P", "Profit USD"]]


In [ ]:
df

In [ ]:
df.info()

In [ ]:
import plotly.express as px


In [ ]:

# Example DataFrame cleanup if not already done
df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
df = df[df["Volume"].notna()]

# Melt the dataframe to show both Open and Close price as individual points
order_df = df
# .melt(
#     id_vars=["Open Time (15m)","Open Time", "Close Time", "Type", "Profit USD"],
#     value_vars=["Open Price", "Close Price"],
#     var_name="Price Type",
#     value_name="Price"
# )

# Use Open Time for both points to align them visually
# df_melted["Time"] = df_melted.apply(
#     lambda row: row["Open Time"] if row["Price Type"] == "Open Price" else row["Close Time"],
#     axis=1
# )

# Color profit: positive = green, negative = red
order_df["Profit Color"] = order_df["Profit USD"].apply(
    lambda x: "green" if x >= 0 else "red")
order_df["Profit Size"] = order_df["Profit USD"].apply(
    lambda x: abs(x))

fig = px.scatter(
    order_df,
    x="Open Time",
    y="Open Price",
    color="Profit USD",
    # color_continuous_scale='YlGn',
    color_continuous_midpoint=0,  # This centers the color scale at 0
    range_color=[-300, 300],           # Set min/max of color range
    # size="Profit Size",
    color_continuous_scale=profit_color_continuous_scale,
    symbol="Type",
    facet_col="Type",
    title="Entry and Exit Prices by Trade",
    hover_data=["Profit USD", "Type"]
)

fig.update_layout(height=500, xaxis_title="Time", yaxis_title="Price",template=template)
fig.show()

In [ ]:
order_df

In [ ]:
order_count_by_close = order_df.groupby("Close Time").size().reset_index(name="Order Count")
order_count_by_close.sort_values('Order Count')

In [ ]:
hist_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']

hist_df = pd.read_csv('/content/drive/MyDrive/algo/input/XAUUSD_M5.csv',names=hist_cols)

In [ ]:
hist_df

In [ ]:

# Step 2: Prepare datetime
hist_df['datetime'] = pd.to_datetime(hist_df['datetime'])

hist_df = hist_df[hist_df['datetime']>=order_df['Open Time'].min()]
hist_df = hist_df[hist_df['datetime']<order_df['Close Time'].max()]

# Step 3: Plotting
fig = go.Figure(data=[go.Candlestick(
    x=hist_df['datetime'],
    open=hist_df['open'],
    high=hist_df['high'],
    low=hist_df['low'],
    close=hist_df['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

fig.update_layout(
    title='Gold Spot Price (15min intervals)',
    xaxis_title='Time',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,
    template=template
)

fig.show()

In [ ]:
ta_df = dropna(hist_df)
ta_df['datetime'] = pd.to_datetime(ta_df['datetime'], errors='coerce')
ta_df = ta_df.dropna(subset=['datetime'])

# Add ta features filling NaN values
ta_df = add_all_ta_features(
    ta_df, open="open", high="high", low="low", close="close", volume="volume", fillna=True)

In [ ]:
for period in [10, 20, 50, 100, 200]:
    ta_df[f'ema_{period}'] = ta.trend.EMAIndicator(close=ta_df['close'], window=period, fillna=True).ema_indicator()

# Add RSI for common periods
for period in [14, 21]:
    ta_df[f'rsi_{period}'] = ta.momentum.RSIIndicator(close=ta_df['close'], window=period, fillna=True).rsi()

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from ta.volatility import BollingerBands
# Create subplots
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[0.7, 0.3],
    subplot_titles=("Price with EMAs (OHLC) and Trades", "RSI")
)

# --- OHLC Chart ---
fig.add_trace(go.Candlestick(
    x=ta_df['datetime'],
    open=ta_df['open'],
    high=ta_df['high'],
    low=ta_df['low'],
    close=ta_df['close'],
    # increasing_line_color='green',
    # decreasing_line_color='red',

), row=1, col=1)
fig.update_layout(xaxis_rangeslider_visible=False)
# --- EMA Lines ---
for period in [50, 200]:
    fig.add_trace(go.Scatter(
        x=ta_df['datetime'],
        y=ta_df[f'ema_{period}'],
        mode='lines',
        name=f'EMA {period}',
        connectgaps=False,
    ), row=1, col=1)

# --- Custom Trade Markers: "buy" and "sell" ---
for trade_type, symbol in [("buy", "arrow-up"), ("sell", "arrow-down")]:
    trade_subset = order_df[order_df["Type"].str.lower() == trade_type]
    fig.add_trace(go.Scatter(
        x=trade_subset["Open Time"],
        y=trade_subset["Open Price"],
        mode="markers",
        name=f"{trade_type.title()} Highlight",
        marker=dict(
            symbol=symbol,
            size=16,  # Slightly bigger than main
            color=trade_subset["Close Time"].apply(color_from_time),  # Color by Close Time
            # opacity=0.3,
            line=dict(width=0)  # No border
        ),
        showlegend=False,  # Hide from legend
        hoverinfo='skip'   # No hover for background layer
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=trade_subset["Open Time"],
                # x=trade_subset["Open Time (15m)"],
        y=trade_subset["Open Price"],
        mode="markers",
        name=f"{trade_type.title()} Trade",
        connectgaps=False,
        marker=dict(
            symbol=symbol,
            size=12,
            color=trade_subset["Profit USD"],  # Color by profit
            colorscale=profit_color_continuous_scale,  # Optional custom scale
            cmin=-300,  # Set these based on your profit range
            cmax=300,
            colorbar=dict(title="Profit USD"),
            line=dict(width=1, color='black')
        ),
        hovertemplate=(
            "Time: %{x}<br>" +
            "Price: %{y}<br>" +
            "Profit USD: %{customdata[0]}<br>" +
                        "Close Time: %{customdata[2]}<br>" +

            "Type: %{customdata[1]}"
        ),
        customdata=trade_subset[["Profit USD", "Type","Close Time"]].values
    ), row=1, col=1)
#Open and Close line
for trade_type in ["buy", "sell"]:
    trade_subset = order_df[order_df["Type"].str.lower() == trade_type]

    for _, row in trade_subset.iterrows():
        fig.add_trace(go.Scatter(
            x=[row["Open Time"], row["Close Time"]],
            y=[row["Open Price"], row["Close Price"]],
            mode="lines",
            line=dict(
                color="gray" if trade_type == "buy" else "darkred",
                width=1,
                dash="dot"
            ),
            showlegend=False,
            hoverinfo="skip"
        ), row=1, col=1)

# --- RSI Line ---
fig.add_trace(go.Scatter(
        x=ta_df['datetime'],
    y=ta_df['rsi_14'],
    mode='lines',
    name='RSI 14',
    line=dict(color='orange'),
    connectgaps=False,
), row=2, col=1)

# # RSI Overbought/Oversold Lines
start_date = ta_df['datetime'].iloc[0]
end_date = ta_df['datetime'].iloc[-1]

fig.add_shape(type="line", x0=start_date, x1=end_date, y0=70, y1=70,
              line=dict(dash='dash', color='red'), row=2, col=1)
fig.add_shape(type="line", x0=start_date, x1=end_date, y0=30, y1=30,
              line=dict(dash='dash', color='green'), row=2, col=1)
overbought = ta_df[ta_df['rsi_14'] > 70]
oversold = ta_df[ta_df['rsi_14'] < 30]


bb_indicator = BollingerBands(close=ta_df["close"], window=20, window_dev=2)

# Add bands to the DataFrame
ta_df["bb_upper"] = bb_indicator.bollinger_hband()
ta_df["bb_middle"] = bb_indicator.bollinger_mavg()
ta_df["bb_lower"] = bb_indicator.bollinger_lband()
# --- Bollinger Bands ---
fig.add_trace(go.Scatter(
    x=ta_df['datetime'],
    y=ta_df['bb_upper'],
    mode='lines',
    name='BB Upper',
    line=dict(color='lightblue', width=1),
    opacity=0.6,
    connectgaps=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=ta_df['datetime'],
    y=ta_df['bb_lower'],
    mode='lines',
    name='BB Lower',
    line=dict(color='lightblue', width=1),
    opacity=0.6,
    fill='tonexty',  # Fill between upper and lower
    connectgaps=False
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=ta_df['datetime'],
    y=ta_df['bb_middle'],
    mode='lines',
    name='BB Middle',
    line=dict(color='blue', width=1),
    connectgaps=False
), row=1, col=1)
# --- Bollinger Bands ---

# Shade overbought regions on main chart
for dt in overbought['datetime']:
    fig.add_vrect(
        x0=dt, x1=dt,
        fillcolor="red", opacity=0.2,
        layer="below", line_width=0,
        row=1, col=1
    )

# Shade oversold regions on main chart
for dt in oversold['datetime']:
    fig.add_vrect(
        x0=dt, x1=dt,
        fillcolor="green", opacity=0.2,
        layer="below", line_width=0,
        row=1, col=1
    )
# Layout Settings
fig.update_layout(
    # title='Stock OHLC with EMA, Trade Entries (Buy/Sell), and RSI',
    height=800,
    legend=dict(x=0, y=1.1, orientation='h'),
    xaxis2_title='Date',
    yaxis1_title='Price',
    yaxis2_title='RSI',
    template=template
)



# from datetime import datetime

# # 1. Get all unique Close Times (as strings for dropdown labels)
# unique_close_times = sorted(df_melted["Close Time"].dt.strftime("%Y-%m-%d %H:%M:%S").unique())
# all_trace_count = len(fig.data)

# # 2. Create dropdown buttons
# buttons = []

# # "All" button to show everything
# buttons.append(dict(
#     label="All",
#     method="update",
#     args=[{"visible": [True] * all_trace_count},
#           {"title": "All Trades"}]
# ))

# # Add one button per Close Time
# for close_time_str in unique_close_times:
#     visible = []
#     for trace in fig.data:
#         # Check if this trace's x matches only that Close Time
#         trace_x = list(trace.x)
#         if len(trace_x) == 0 or not isinstance(trace_x[0], datetime):
#             visible.append(True)  # Show all non-trade traces
#         else:
#             # Filter only trades for this Close Time
#             match = any(
#                 pd.to_datetime(close_time_str) == pd.to_datetime(row["Close Time"])
#                 for i, row in df_melted.iterrows()
#                 if row["Time"] in trace_x
#             )
#             visible.append(match)

#     buttons.append(dict(
#         label=close_time_str,
#         method="update",
#         args=[{"visible": visible},
#               {"title": f"Trades for Close Time: {close_time_str}"}]
#     ))

# # 3. Add dropdown to layout
# fig.update_layout(
#     updatemenus=[
#         dict(
#             active=0,
#             buttons=buttons,
#             x=1.02,
#             y=1.1,
#             xanchor="left",
#             yanchor="top",
#             direction="down",
#             showactive=True
#         )
#     ]
# )


fig.show()


In [ ]:
ta_df[ta_df['datetime'].isna()]

In [ ]:
df_melted[df_melted['Time'].isna()]